In [1]:
# tab data
import pandas as pd
import numpy as np

# data and prep
from wrangle import wrangle_zillow
from preprocess import scale_data

# data viz
import matplotlib.pyplot as plt

# stats and modeling needs
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [2]:
train, validate, test = wrangle_zillow()

In [3]:
# get only oc for each
train = train[train.county == 'Orange']
# get only oc for each
validate = validate[validate.county == 'Orange']
# get only oc for each
test = test[test.county == 'Orange']

In [4]:
num_cols = ['square_feet', 'beds']

In [5]:
# xtrain, ytrain
X_train = train[num_cols]
y_train = train[['tax_value']]

# xvalidate, yvalidate
X_validate = validate[num_cols]
y_validate = validate[['tax_value']]

# xtest, ytest
X_test = test[num_cols]
y_test = test[['tax_value']]

In [6]:
# scaling
X_train_scaled, X_validate_scaled, X_test_scaled = scale_data(X_train, num_cols, 'minmax', X_validate=X_validate, X_test=X_test)

In [7]:
y='tax_value'
# baseline using median
y_train['baseline_mean'] = round(y_train[y].mean(), 2)
y_validate['baseline_mean'] = round(y_validate.loc[:,y].mean(), 2)

In [8]:
# RMSE of mean baseline predictions
rmse_train = mean_squared_error(y_train.loc[:,y], y_train['baseline_mean'], squared=False)
rmse_validate = mean_squared_error(y_validate.loc[:,y], y_validate['baseline_mean'], squared=False)

# checking scores
rmse_train, rmse_validate, rmse_train - rmse_validate

(259532.8366898489, 255477.65091251553, 4055.1857773333613)

In [9]:
def basic_linear_modeling(X_train, y_train, x_validate, y_validate, model):
    '''
    Arguments:
    Actions:
    Returns:
    Modules:
    '''
    # fit the model
    model.fit(X_train, y_train)
   
    # return predictions
    return model.predict(X_train), model.predict(X_validate)
    

In [10]:
y_train['ols_preds'], y_validate['ols_preds'] = basic_linear_modeling(X_train_scaled, y_train[['tax_value']], X_validate_scaled, y_validate[['tax_value']], model=LinearRegression())

In [11]:
y_train['lars_alpha_hundredth_preds'], y_validate['lars_alpha_tenth_preds'] = basic_linear_modeling(X_train_scaled, y_train[['tax_value']], X_validate_scaled, y_validate[['tax_value']], model=LassoLars(alpha=.01))
y_train['lars_alpha_tenth_preds'], y_validate['lars_alpha_tenth_preds'] = basic_linear_modeling(X_train_scaled, y_train[['tax_value']], X_validate_scaled, y_validate[['tax_value']], model=LassoLars(alpha=.1))
y_train['lars_alpha_half_preds'], y_validate['lars_alpha_half_preds'] = basic_linear_modeling(X_train_scaled, y_train[['tax_value']], X_validate_scaled, y_validate[['tax_value']], model=LassoLars(alpha=.5))
y_train['lars_alpha1_preds'], y_validate['lars_alpha1_preds'] = basic_linear_modeling(X_train_scaled, y_train[['tax_value']], X_validate_scaled, y_validate[['tax_value']], model=LassoLars(alpha=1))

In [12]:
y_train['glm_poi_preds'], y_validate['glm_poi_preds'] = basic_linear_modeling(X_train_scaled, y_train[['tax_value']], X_validate_scaled, y_validate[['tax_value']], model=TweedieRegressor(power=1))

In [13]:
# make the polyni mial features
pf = PolynomialFeatures(degree=2)

# fit_transform
X_train_degree2 = pf.fit_transform(X_train_scaled)

# transform for validate
X_validate_degree2 = pf.transform(X_validate_scaled)


In [14]:
# create the model object
lm2 = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm2.fit(X_train_degree2, y_train.tax_value)

# predict train
y_train['poly_preds'] = lm2.predict(X_train_degree2)


# predict validate
y_validate['poly_preds'] = lm2.predict(X_validate_degree2)


In [15]:
y_train.columns

Index(['tax_value', 'baseline_mean', 'ols_preds', 'lars_alpha_hundredth_preds',
       'lars_alpha_tenth_preds', 'lars_alpha_half_preds', 'lars_alpha1_preds',
       'glm_poi_preds', 'poly_preds'],
      dtype='object')

In [16]:
model_rmse = []
target = 'tax_value'
for col in y_train:
    if col in ['glm_poi_preds','lars_alpha_hundredth_preds', 'tax_value']:
        pass
    else:
        rmse_train = mean_squared_error(y_train[target], round(y_train[col], 4), squared=False)
        rmse_validate = mean_squared_error(y_validate[target], round(y_validate[col], 4), squared=False)
        rmse_diff = rmse_train - rmse_validate
        model_rmse.append({
            'Model': col,
            'Train RMSE': rmse_train,
            'Validate RMSE': rmse_validate,
            'RMSE Difference': rmse_diff
        })

In [17]:
train = 1000000
for i in model_rmse:
    if i['Train RMSE'] < train:
        train = i['Train RMSE']

# getting the lowest train RMSE
for i in model_rmse:
    if i['Train RMSE'] == train:
        print(i)

{'Model': 'poly_preds', 'Train RMSE': 225923.04294255623, 'Validate RMSE': 225973.2523783483, 'RMSE Difference': -50.209435792057775}


In [18]:
model_rmse

[{'Model': 'baseline_mean',
  'Train RMSE': 259532.8366898489,
  'Validate RMSE': 255477.65091251553,
  'RMSE Difference': 4055.1857773333613},
 {'Model': 'ols_preds',
  'Train RMSE': 227111.28182722794,
  'Validate RMSE': 1285167855.3561206,
  'RMSE Difference': -1284940744.0742934},
 {'Model': 'lars_alpha_tenth_preds',
  'Train RMSE': 227111.28222588135,
  'Validate RMSE': 1285041247.639511,
  'RMSE Difference': -1284814136.3572853},
 {'Model': 'lars_alpha_half_preds',
  'Train RMSE': 227111.29178338894,
  'Validate RMSE': 1284534816.7730932,
  'RMSE Difference': -1284307705.48131},
 {'Model': 'lars_alpha1_preds',
  'Train RMSE': 227111.32165206224,
  'Validate RMSE': 1283901778.1901119,
  'RMSE Difference': -1283674666.8684597},
 {'Model': 'poly_preds',
  'Train RMSE': 225923.04294255623,
  'Validate RMSE': 225973.2523783483,
  'RMSE Difference': -50.209435792057775}]